# Labo pratique : Arbres de décision

Dans cet exercice, vous allez implémenter un arbre de décision à partir de zéro et l'appliquer à la tâche de classification d'un champignon pour savoir s'il est comestible ou vénéneux.

# Outline
- [ 1 - Packages ](#1)
- [ 2 -  Problem Statement](#2)
- [ 3 - Dataset](#3)
  - [ 3.1 One hot encoded dataset](#3.1)
- [ 4 - Decision Tree Refresher](#4)
  - [ 4.1  Calculate entropy](#4.1)
    - [ Exercise 1](#ex01)
  - [ 4.2  Split dataset](#4.2)
    - [ Exercise 2](#ex02)
  - [ 4.3  Calculate information gain](#4.3)
    - [ Exercise 3](#ex03)
  - [ 4.4  Get best split](#4.4)
    - [ Exercise 4](#ex04)
- [ 5 - Building the tree](#5)


<a name="1"></a>
## 1 - Packages 

Tout d'abord, exécutons la cellule ci-dessous pour importer tous les paquets dont vous aurez besoin au cours de ce travail.
- [numpy](https://www.numpy.org) est le paquetage fondamental pour travailler avec des matrices en Python.
- [matplotlib](https://matplotlib.org) est une bibliothèque célèbre pour tracer des graphiques en Python.
- ``utils.py`` contient des fonctions d'aide pour ce travail. Vous n'avez pas besoin de modifier le code de ce fichier.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *

%matplotlib inline

<a name="2"></a>
## 2 - Énoncé du problème

Supposons que vous créiez une entreprise qui cultive et vend des champignons sauvages. 
- Comme tous les champignons ne sont pas comestibles, vous aimeriez pouvoir déterminer si un champignon donné est comestible ou vénéneux sur la base de ses caractéristiques physiques
- Vous disposez de données existantes que vous pouvez utiliser pour cette tâche. 

Pouvez-vous utiliser ces données pour vous aider à identifier les champignons qui peuvent être vendus en toute sécurité ? 

Remarque : l'ensemble de données utilisé ne l'est qu'à des fins d'illustration. Il ne s'agit pas d'un guide d'identification des champignons comestibles.

- Vous disposez de 10 exemples de champignons. Pour chaque exemple, vous disposez de
    - Trois caractéristiques
        - Couleur du chapeau (`Brun` ou `Red`),
        - Forme du pied (`Tapissement` ou `Agrandissement`), et
        - Solitaire (`Oui` ou `Non`)
    - Étiquette
        - Comestible (`1` indiquant oui ou `0` indiquant toxique)


<a name="3"></a>
## 3 - Dataset

Vous commencerez par charger l'ensemble de données pour cette tâche. L'ensemble de données que vous avez collecté est le suivant :


3.1 Ensemble de données codées à chaud
Pour faciliter la mise en œuvre, nous avons codé les caractéristiques à chaud (nous les avons transformées en caractéristiques à valeur 0 ou 1).

| Cap Color | Stalk Shape | Solitary | Edible |
|:---------:|:-----------:|:--------:|:------:|
|   Brown   |   Tapering  |    Yes   |    1   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |   Tapering  |    Yes   |    1   |
|    Red    |   Tapering  |    Yes   |    0   |
|    Red    |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|    Red    |   Tapering  |    No    |    1   |
|   Brown   |  Enlarging  |    No    |    0   |

Par conséquent, `X_train` contient trois caractéristiques pour chaque exemple,
- `X_train` contient trois caractéristiques pour chaque exemple 
    - Couleur brune (une valeur de `1` indique une couleur de chapeau "brune" et `0` indique une couleur de chapeau "rouge")
    - Forme effilée (une valeur de `1` indique une forme de tige effilée et une valeur de `0` indique une forme de tige élargie)
    - Solitaire (Une valeur de `1` indique "Oui" et `0` indique "Non")

- `y_train` indique si le champignon est comestible 
    - `y = 1` indique qu'il est comestible
    - `y = 0` indique qu'il est toxique

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

#### Afficher les variables
Familiarisez-vous avec votre ensemble de données.  
- Un bon point de départ est d'imprimer chaque variable et de voir ce qu'elle contient.

Le code ci-dessous affiche les premiers éléments de `X_train` et le type de la variable.

In [3]:
print("First few elements of X_train:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Type of X_train: <class 'numpy.ndarray'>


Maintenant, faisons la même chose pour `y_train`

In [4]:
print("First few elements of y_train:", y_train[:5])
print("Type of y_train:",type(y_train))

First few elements of y_train: [1 1 0 0 1]
Type of y_train: <class 'numpy.ndarray'>


#### Vérifiez les dimensions de vos variables

Une autre façon utile de se familiariser avec vos données est de visualiser leurs dimensions.

Veuillez imprimer la forme de `X_train` et `y_train` et voir combien d'exemples de formation vous avez dans votre jeu de données.

In [5]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


<a name="4"></a>
## 4 - Remise à niveau sur les arbres de décision

Dans ce TP, vous allez construire un arbre de décision basé sur le jeu de données fourni.

- Rappelons que les étapes de construction d'un arbre de décision sont les suivantes :
    - Commencer avec tous les exemples au nœud racine
    - Calculer le gain d'information pour la division sur toutes les caractéristiques possibles, et choisir celle qui présente le gain d'information le plus élevé.
    - Diviser l'ensemble de données en fonction de la caractéristique sélectionnée et créer les branches gauche et droite de l'arbre.
    - Répéter le processus de fractionnement jusqu'à ce que les critères d'arrêt soient remplis.
  
  
- Dans ce laboratoire, vous mettrez en œuvre les fonctions suivantes, qui vous permettront de diviser un nœud en branches gauche et droite à l'aide de la caractéristique présentant le gain d'information le plus élevé
    - Calculer l'entropie au niveau d'un noeud 
    - Diviser l'ensemble de données au niveau d'un noeud en branches gauche et droite sur la base d'une caractéristique donnée
    - Calculer le gain d'information résultant de la division sur la base d'une caractéristique donnée
    - Choisir la caractéristique qui maximise le gain d'information
    
- Nous utiliserons ensuite les fonctions d'aide que vous avez implémentées pour construire un arbre de décision en répétant le processus de division jusqu'à ce que le critère d'arrêt soit satisfait. 
    - Pour ce laboratoire, le critère d'arrêt que nous avons choisi est la définition d'une profondeur maximale de 2.

<a name="4.1"></a>
### 4.1  Calculer l'entropie

Tout d'abord, vous allez écrire une fonction d'aide appelée `compute_entropy` qui calcule l'entropie (mesure de l'impureté) d'un noeud. 
- La fonction prend un tableau numpy (`y`) qui indique si les exemples de ce nœud sont comestibles (`1`) ou toxiques (`0`). 

Complétez la fonction `compute_entropy()` ci-dessous pour :
* Calculer $p_1$, qui est la fraction d'exemples qui sont comestibles (c'est-à-dire qui ont une valeur = `1` dans `y`).
* L'entropie est alors calculée comme suit 

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$
* Note 
    * The log is calculated with base $2$
    * For implementation purposes, $0\text{log}_2(0) = 0$. That is, if `p_1 = 0` or `p_1 = 1`, set the entropy to `0`
    * Make sure to check that the data at a node is not empty (i.e. `len(y) != 0`). Return `0` if it is
    
<a name="ex01"></a>
### Exercise 1
Veuillez compléter la fonction `compute_entropy()` en utilisant les instructions précédentes.
    
Si vous êtes bloqué, vous pouvez consulter les conseils présentés après la cellule ci-dessous pour vous aider dans l'implémentation.

In [6]:
# UNQ_C1
# GRADED FUNCTION: compute_entropy

def compute_entropy(y):
    """
    Computes the entropy for 
    
    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)
       
    Returns:
        entropy (float): Entropy at that node
        
    """
    # You need to return the following variables correctly
    entropy = 0.
    
    ### START CODE HERE ###
    if len(y) != 0:
        p1 = p1 = len(y[y == 1]) / len(y) 
     # For p1 = 0 and 1, set the entropy to 0 (to handle 0log0)
        if p1 != 0 and p1 != 1:
             entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
        else:
             entropy = 0
    ### END CODE HERE ###        
    
    return entropy

Vous pouvez vérifier si votre mise en œuvre est correcte en exécutant le code de test suivant :

In [7]:
# Compute entropy at the root node (i.e. with all examples)
# Since we have 5 edible and 5 non-edible mushrooms, the entropy should be 1"

print("Entropy at root node: ", compute_entropy(y_train)) 

# UNIT TESTS
compute_entropy_test(compute_entropy)

Entropy at root node:  1.0
 All tests passed.


**Expected Output**:
<table>
  <tr>
    <td> <b>Entropy at root node:<b> 1.0 </td> 
  </tr>
</table>

<a name="4.2"></a>
### 4.2  Diviser le jeu de données

Ensuite, vous écrirez une fonction d'aide appelée `split_dataset` qui prend les données à un noeud et une caractéristique à diviser et la divise en branches gauche et droite. Plus tard dans le labo, vous implémenterez un code pour calculer la qualité de la division.

- La fonction prend en charge les données d'apprentissage, la liste des indices des points de données à ce noeud, ainsi que la caractéristique à diviser. 
- Elle divise les données et renvoie le sous-ensemble d'indices à la branche gauche et à la branche droite.
- Par exemple, disons que nous commençons au noeud racine (donc `node_indices = [0,1,2,3,4,5,6,7,8,9]`), et que nous choisissons de diviser sur la caractéristique `0`, qui est de savoir si l'exemple a une casquette marron ou non.
    - La sortie de la fonction est donc `indices_gauche = [0,1,2,3,4,7,9]` et `indices_droite = [5,6,8]`
    
| Index | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:-----:|:---------:|:--------------------:|:--------:|:------:|
|   0   |     1     |           1          |     1    |    1   |
|   1   |     1     |           0          |     1    |    1   |
|   2   |     1     |           0          |     0    |    0   |
|   3   |     1     |           0          |     0    |    0   |
|   4   |     1     |           1          |     1    |    1   |
|   5   |     0     |           1          |     1    |    0   |
|   6   |     0     |           0          |     0    |    0   |
|   7   |     1     |           0          |     1    |    1   |
|   8   |     0     |           1          |     0    |    1   |
|   9   |     1     |           0          |     0    |    0   |

<a name="ex02"></a>
### Exercise 2

Veuillez compléter la fonction `split_dataset()` présentée ci-dessous

- Pour chaque index dans `node_indices`
    - Si la valeur de `X` à cet index pour cet élément est `1`, ajoutez l'index à `left_indices`
    - Si la valeur de `X` à cet index pour cet élément est `0`, ajoutez l'index à `right_indices`

Si vous êtes bloqué, vous pouvez consulter les conseils présentés après la cellule ci-dessous pour vous aider dans l'implémentation.

In [8]:
# UNQ_C2
# GRADED FUNCTION: split_dataset

def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):  List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list): Indices with feature value == 1
        right_indices (list): Indices with feature value == 0
    """
    
    # You need to return the following variables correctly
    left_indices = []
    right_indices = []
    
    ### START CODE HERE ###
    for i in node_indices:   
        if X[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    ### END CODE HERE ###
        
    return left_indices, right_indices

Vérifions maintenant votre mise en œuvre à l'aide des blocs de code ci-dessous. Essayons de diviser l'ensemble de données au niveau du nœud racine, qui contient tous les exemples de la caractéristique 0 (casquette brune), comme nous l'avons vu plus haut.

In [9]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# UNIT TESTS    
split_dataset_test(split_dataset)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
 All tests passed.


**Expected Output**:
```
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
```

<a name="4.3"></a>
### 4.3  Calculer le gain d'information

Ensuite, vous écrirez une fonction appelée `information_gain` qui prend les données d'apprentissage, les indices d'un noeud et une caractéristique à diviser et renvoie le gain d'information de la division.

<a name="ex03"></a>
### Exercise 3

Veuillez compléter la fonction `compute_information_gain()` présentée ci-dessous pour calculer

$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

où 
- $H(p_1^\text{node})$ est l'entropie au niveau du nœud 
- $H(p_1^\text{gauche})$ et $H(p_1^\text{droite})$ sont les entropies des branches gauche et droite résultant de la scission
- $w^{\text{left}}$ et $w^{\text{right}}$ sont les proportions d'exemples dans les branches gauche et droite, respectivement.

Remarque :
- Vous pouvez utiliser la fonction `compute_entropy()` que vous avez implémentée ci-dessus pour calculer l'entropie.
- Nous avons fourni un code de démarrage qui utilise la fonction `split_dataset()` que vous avez implémentée ci-dessus pour diviser l'ensemble de données. 

Si vous êtes bloqué, vous pouvez consulter les conseils présentés après la cellule ci-dessous pour vous aider dans l'implémentation.

In [10]:
# UNQ_C3
# GRADED FUNCTION: compute_information_gain

def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    ### START CODE HERE ###
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)
    
    # Weights 
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    
    #Weighted entropy
    weighted_entropy = w_left * left_entropy + w_right * right_entropy
    
    #Information gain 
    information_gain = node_entropy - weighted_entropy
    
    ### END CODE HERE ###  
    
    return information_gain

Vous pouvez maintenant vérifier votre mise en œuvre à l'aide de la cellule ci-dessous et calculer le gain d'information qu'apporterait une répartition sur chacune des caractéristiques.

In [11]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)
    
info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

# UNIT TESTS
compute_information_gain_test(compute_information_gain)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
 All tests passed.


**Expected Output**:
```
Informations obtenues en divisant la racine sur le chapeau brun :  0.034851554559677034
Gain d'information de la division de la racine sur la forme effilée de la tige : 0.12451124978365313
Gain d'information lié à la division de la racine sur la forme solitaire :  0.2780719051126377
```

Le fractionnement sur "Solitaire" (caractéristique = 2) au nœud racine donne le gain d'information maximal. Il s'agit donc de la meilleure caractéristique sur laquelle effectuer le découpage au niveau du nœud racine.

<a name="4.4"></a>
### 4.4  Obtenir la meilleure répartition
Ecrivons maintenant une fonction pour obtenir la meilleure caractéristique à diviser en calculant le gain d'information de chaque caractéristique comme nous l'avons fait ci-dessus et en renvoyant la caractéristique qui donne le gain d'information maximal.

<a name="ex04"></a>
### Exercise 4
Veuillez compléter la fonction `get_best_split()` présentée ci-dessous.
- La fonction prend en compte les données d'apprentissage, ainsi que les indices des points de données à ce noeud
- La sortie de la fonction est la caractéristique qui donne le gain d'information maximal. 
    - Vous pouvez utiliser la fonction `compute_information_gain()` pour itérer à travers les caractéristiques et calculer l'information pour chaque caractéristique.
Si vous êtes bloqué, vous pouvez consulter les conseils présentés après la cellule ci-dessous pour vous aider dans la mise en œuvre.

In [12]:
# UNQ_C4
# GRADED FUNCTION: get_best_split

def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    
    ### START CODE HERE ###
    max_info_gain=0
    for feature in range(num_features):
        info_gain = compute_information_gain(X, y, node_indices, feature)
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_feature = feature
                
        
    ### END CODE HERE ##    
       
    return best_feature


Maintenant, vérifions la mise en œuvre de votre fonction à l'aide de la cellule ci-dessous.

In [13]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

# UNIT TESTS
get_best_split_test(get_best_split)

Best feature to split on: 2
 All tests passed.


Comme nous l'avons vu plus haut, la fonction renvoie que la meilleure caractéristique à scinder au nœud racine est la caractéristique 2 ("Solitaire").

<a name="5"></a>
## 5 - Construction de l'arbre

Dans cette section, nous utilisons les fonctions que vous avez mises en œuvre ci-dessus pour générer un arbre de décision en choisissant successivement la meilleure caractéristique à scinder jusqu'à ce que nous atteignions le critère d'arrêt (la profondeur maximale est de 2).

Vous n'avez pas besoin d'implémenter quoi que ce soit pour cette partie.

In [14]:
# Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    tree.append((current_depth, branch_name, best_feature, node_indices))
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)


In [15]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
